<a href="https://colab.research.google.com/github/sudhakar2003/Build-a-Music-Recommendation-System/blob/main/Build_a_Music_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import numpy as np
import pandas as pd
from typing import List, Dict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [62]:
songs = pd.read_csv("/content/songdata.csv")

In [63]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [64]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [65]:
songs['text'] = songs['text'].str.replace(r'\n', '')

<ipython-input-65-215fe46be90d>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n', '')


In [66]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [67]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

In [68]:
cosine_similarities = cosine_similarity(lyrics_matrix)

In [69]:
similarities = {}

In [70]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs.
    similar_indices = cosine_similarities[i].argsort()[:-50:-1]
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [71]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)

        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score")
            print("--------------------")

    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [72]:
recommedations = ContentBasedRecommender(similarities)

In [73]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4
}

In [74]:
recommedations.recommend(recommendation)

The 4 recommended songs for I Can Wait Forever are:
Number 1:
Are You Gonna Wait Forever? by U2 with 0.47 similarity score
--------------------
Number 2:
Always And Forever by Kenny Rogers with 0.384 similarity score
--------------------
Number 3:
Forever In Your Heart by 'n Sync with 0.375 similarity score
--------------------
Number 4:
Tell Me You'll Wait For Me by Ray Charles with 0.368 similarity score
--------------------


In [75]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4
}

In [76]:
recommedations.recommend(recommendation2)

The 4 recommended songs for The Mention Of Your Name are:
Number 1:
Lonely Days by Bee Gees with 0.208 similarity score
--------------------
Number 2:
Friend Of A Friend by Tim McGraw with 0.195 similarity score
--------------------
Number 3:
Friends by Nazareth with 0.186 similarity score
--------------------
Number 4:
Conceal Me by Xavier Rudd with 0.18 similarity score
--------------------
